**Description of Parameters**

Standard Columns (known before race)


* AgeRestriction: Age of horse allowed to participate in the race
* Barrier: Position in the starting stall, 0 being the inner most lane
* ClassRestriction:
* CourseIndicator:
* Dam ID: ID of the mother of the horse
* Distance: Length of the race in metres
* Foaling Country: The country the horse was born in
* Foaling Date: The day the horse was born
* FrontShoes:
* Gender:
* GoingAbbrey
* GoingID
* HandicapDistance
* HandicapType
* HindShoes:
* HorseAge
* HorseID
* JockeyID
* RaceGroup
* RaceID
* RacePrizemoney
* RaceStartTime
* RacingSubType
* Saddlecloth
* SexRestriction: M if male only race, F if female only race
* SireID: ID of the father of the horse
* StartType
* StartingLine
* Surface
* TrackID
* TrainerID
* WeightCarried
* WetnessScale


Performace (result of the race)


*   BeatenMargin: number of horse lengths the horse has been beaten by?
*   Disqualified: True if horse was disqualified, False if it was not. Some examples include:
  * Gallop
  * Trap
  * Aubin
  * Pace
* FinishPosition: Place finished in the race or reason for not placing
  * BS = Break Stride
  * PU = Pulled Up
  * FL = Fell
  * NP = Took no Part

* PIRPosition
* Prizemoney
* RaceOverallTime
* PriceSP
* NoFrontCover
* PositionInRunning
* WideOffRail







In [31]:
# Import Packages
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

In [32]:
# read data file
df = pd.read_parquet("trots_2013-2022.parquet", engine = 'pyarrow')

**Data Cleaning and Preprocessing**


In [33]:
# drop class restriction
df = df.drop(columns=["ClassRestriction"])

In [34]:
# break age restriction into lower age limit and upper age limit
df["LowerAge"] = df["AgeRestriction"].astype(str).str[0]
df["LowerAge"] = df["LowerAge"].replace("P", 9)
df["LowerAge"] = pd.to_numeric(df["LowerAge"])
df["LowerAge"] = df["LowerAge"].fillna(0)
df["UpperAge"] = df["AgeRestriction"].astype(str).str.replace("yo", "").astype(str).str[-1]
df["UpperAge"] = df["UpperAge"].replace('0', 10)
df["UpperAge"] = df["UpperAge"].replace("+", 15)
df["UpperAge"] = pd.to_numeric(df["UpperAge"])
df["UpperAge"] = df["UpperAge"].fillna(20)
df = df.drop(columns = ["AgeRestriction"])

In [35]:
# break race start time into year, month, day, hour, and minute
df["RaceStartYear"] = df["RaceStartTime"].dt.year
df["RaceStartMonth"] = df["RaceStartTime"].dt.month
df["RaceStartDay"] = df["RaceStartTime"].dt.day
df["RaceStartHour"] = df["RaceStartTime"].dt.hour
df["RaceStartMinute"] = df["RaceStartTime"].dt.minute

In [36]:
# break foaling date into year, month, and day
df["FoalingDateYear"] = df["FoalingDate"].dt.year
df["FoalingDateMonth"] = df["FoalingDate"].dt.month
df["FoalingDateDay"] = df["FoalingDate"].dt.day

In [37]:
# drop foaling date column
df = df.drop(columns = ["FoalingDate"])

In [38]:
# break finish position into place and DNF reason
df["DNFReason"] = df["FinishPosition"].astype(str).str.replace("1","").astype(str)
for i in ["2","3","4","5","6","7","8","9","0", " "]:
  df["DNFReason"] = df["DNFReason"].astype(str).str.replace(i,"").astype(str)
for i in ["BS","UN","PU","DQ","FL","NP","UR","WC"]:
  df["FinishPosition"] = df["FinishPosition"].astype(str).str.replace(i,'-1').astype(str)
df["FinishPosition"] = df["FinishPosition"].astype(int)

In [39]:
# add win column
df['Win'] = df["FinishPosition"]
for i in ["2","3","4","5","6","7","8","9","0", "10","11","-1"]:
  df["Win"] = df["Win"].astype(str).str.replace(i,"0").astype(str)
df["Win"] = df["Win"].astype(int)
pd.value_counts(df.Win)

0    1111506
1      88906
Name: Win, dtype: int64

In [40]:
# add dummy variables for categorical variables
df = pd.get_dummies(df,columns=["CourseIndicator", "DNFReason","Disqualified", "FoalingCountry", "Gender", "GoingAbbrev", "HandicapType", "RaceGroup", "RacingSubType", "SexRestriction", "StartType", "Surface"], drop_first = True)

In [41]:
# split data into train and test set
train = df[df["RaceStartTime"] < "2021-11-01 00:00:00"]
test = df[df["RaceStartTime"] >= "2021-11-01 00:00:00"]

In [42]:
# remove RaceStartTime and columns
train = train.drop(columns = ["RaceStartTime"])
test = test.drop(columns = ["RaceStartTime"])

**Model 1 - drops all performance variables**

In [43]:
# gets values for X and y
y_train = train.Win.values
X_train = train.drop(columns = ["Win", "BeatenMargin", 'Disqualified_True', "FinishPosition", 'DNFReason_BS',
       'DNFReason_DQ', 'DNFReason_FL', 'DNFReason_NP', 'DNFReason_PU',
       'DNFReason_UN', 'DNFReason_UR', 'DNFReason_WC', "PIRPosition", "Prizemoney", "RaceOverallTime",
                             "PriceSP" , "NoFrontCover", "PositionInRunning" , "WideOffRail" ]).values
y_test = test.Win.values
X_test = test.drop(columns = ["Win", "BeatenMargin", 'Disqualified_True', "FinishPosition", 'DNFReason_BS',
       'DNFReason_DQ', 'DNFReason_FL', 'DNFReason_NP', 'DNFReason_PU',
       'DNFReason_UN', 'DNFReason_UR', 'DNFReason_WC', "PIRPosition", "Prizemoney", "RaceOverallTime",
                             "PriceSP" , "NoFrontCover", "PositionInRunning" , "WideOffRail" ]).values

In [44]:
# define the logistic regression model
logreg = LogisticRegression(solver='lbfgs', max_iter=10000)
lr = logreg.fit(X_train, y_train)

In [45]:
# predict probabilities for win probability column for train data
train['WinProbability'] = lr.predict_proba(X_train)[:,1]
train['WinProbability'] = train['WinProbability'] / train.groupby('RaceID')['WinProbability'].transform('sum')

In [46]:
# predict probabilities for win probability column for test data
test['WinProbability'] = lr.predict_proba(X_test)[:,1]
test['WinProbability'] = test['WinProbability'] / test.groupby('RaceID')['WinProbability'].transform('sum')

In [47]:
# predict the winner for each race by using the highest win probability
train["PredictedWin"] = 0
train.loc[train.groupby('RaceID')['WinProbability'].transform(max) == train['WinProbability'],"PredictedWin"] = 1
test["PredictedWin"] = 0
test.loc[test.groupby('RaceID')['WinProbability'].transform(max) == test['WinProbability'],"PredictedWin"] = 1

In [48]:
# First, get tp, tn, fp, fn
tp = sum(np.logical_and(test['PredictedWin'] == 1, test['Win'] == 1))
tn = sum(np.logical_and(test['PredictedWin'] == 0, test['Win'] == 0))
fp = sum(np.logical_and(test['PredictedWin'] == 1, test['Win'] == 0))
fn = sum(np.logical_and(test['PredictedWin'] == 0, test['Win'] == 1))

print(f"tp: {tp} tn: {tn} fp: {fp} fn: {fn}")

# Accuracy
acc = (tp + tn) / (tp + tn + fp + fn)

# Precision
precision = tp / (tp + fp)

# Recall
recall = tp / (tp + fn)

# Sensitivity
sensitivity = recall

# Specificity
specificity = tn / (fp + tn)

# Print results
print("Accuracy:",round(acc,3),"Recall:",round(recall,3),"Precision:",round(precision,3),
          "Sensitivity:",round(sensitivity,3),"Specificity:",round(specificity,3))

tp: 261 tn: 24098 fp: 1879 fn: 1882
Accuracy: 0.866 Recall: 0.122 Precision: 0.122 Sensitivity: 0.122 Specificity: 0.928


**Model 2 - put all performance variable to 0 in test set**

In [49]:
# get values for X and y
columns_to_zero = ["BeatenMargin", 'Disqualified_True', "FinishPosition", 'DNFReason_BS',
       'DNFReason_DQ', 'DNFReason_FL', 'DNFReason_NP', 'DNFReason_PU',
       'DNFReason_UN', 'DNFReason_UR', 'DNFReason_WC', "PIRPosition", "Prizemoney", "RaceOverallTime",
                             "PriceSP" , "NoFrontCover", "PositionInRunning" , "WideOffRail"]
train2 = train
test2 = test

test2.loc[:, columns_to_zero] = 0

y_train = train.Win.values
X_train = train2.drop(columns = ["Win"]).values
y_test = test.Win.values
X_test = test2.drop(columns = ["Win"]).values

<ipython-input-49-d542ea0a7c94>:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  test2.loc[:, columns_to_zero] = 0


In [50]:
# define the logistic regression model
logreg2 = LogisticRegression(solver='lbfgs', max_iter=10000)
lr2 = logreg2.fit(X_train, y_train)

In [51]:
# predict probabilities for win probability column for train data
train2['WinProbability'] = lr2.predict_proba(X_train)[:,1]
train2['WinProbability'] = train2['WinProbability'] / train2.groupby('RaceID')['WinProbability'].transform('sum')

In [52]:
# predict probabilities for win probability column for test data
test2['WinProbability'] = lr2.predict_proba(X_test)[:,1]
test2['WinProbability'] = test2['WinProbability'] / test2.groupby('RaceID')['WinProbability'].transform('sum')

In [53]:
# predict the winner for each race by using the highest win probability
train2["PredictedWin"] = 0
train2.loc[train2.groupby('RaceID')['WinProbability'].transform(max) == train2['WinProbability'],"PredictedWin"] = 1
test2["PredictedWin"] = 0
test2.loc[test2.groupby('RaceID')['WinProbability'].transform(max) == test2['WinProbability'],"PredictedWin"] = 1

In [54]:
# First, get tp, tn, fp, fn
tp = sum(np.logical_and(test2['PredictedWin'] == 1, test2['Win'] == 1))
tn = sum(np.logical_and(test2['PredictedWin'] == 0, test2['Win'] == 0))
fp = sum(np.logical_and(test2['PredictedWin'] == 1, test2['Win'] == 0))
fn = sum(np.logical_and(test2['PredictedWin'] == 0, test2['Win'] == 1))

print(f"tp: {tp} tn: {tn} fp: {fp} fn: {fn}")

# Accuracy
acc = (tp + tn) / (tp + tn + fp + fn)

# Precision
precision = tp / (tp + fp)

# Recall
recall = tp / (tp + fn)

# Sensitivity
sensitivity = recall

# Specificity
specificity = tn / (fp + tn)

# Print results
print("Accuracy:",round(acc,3),"Recall:",round(recall,3),"Precision:",round(precision,3),
          "Sensitivity:",round(sensitivity,3),"Specificity:",round(specificity,3))

tp: 238 tn: 24075 fp: 1902 fn: 1905
Accuracy: 0.865 Recall: 0.111 Precision: 0.111 Sensitivity: 0.111 Specificity: 0.927


**Model 1 ran better**

We chose model 1 since it had higher accuracy, recall, precision, sensitivity and specificity than model 2.



In [55]:
# combine train and test (used model 1)
final_data = pd.concat([train, test])
final_data = final_data[["HorseID", "RaceID", "WinProbability"]]
final_data

,HorseID,RaceID,WinProbability
0,1514407,1596364,3.201728e-04
1,1509393,1596364,2.441597e-11
2,1507968,1596364,7.196308e-14
3,1508537,1596364,1.020950e-14
4,1514056,1596364,1.084446e-14
...,...,...,...
1192908,1562350,1683313,7.070688e-02
1192909,1560704,1683313,7.131926e-02
1192910,1565306,1683313,7.009350e-02
1192911,1561771,1683313,7.355970e-02


In [56]:
# write the final data that contains the win probability column to a parquet file
final_data.to_parquet('final_data.parquet')